In [1]:
import os
import rasterio as rio
import glob
import shutil

In [25]:
!gcloud storage ls gs://delos-downscale/data/rtma/

gs://delos-downscale/data/rtma/001_038/
gs://delos-downscale/data/rtma/001_039/
gs://delos-downscale/data/rtma/001_040/
gs://delos-downscale/data/rtma/001_041/
gs://delos-downscale/data/rtma/001_042/
gs://delos-downscale/data/rtma/001_043/
gs://delos-downscale/data/rtma/001_044/
gs://delos-downscale/data/rtma/001_045/
gs://delos-downscale/data/rtma/001_046/
gs://delos-downscale/data/rtma/002_034/
gs://delos-downscale/data/rtma/002_035/
gs://delos-downscale/data/rtma/002_036/
gs://delos-downscale/data/rtma/002_037/
gs://delos-downscale/data/rtma/002_038/
gs://delos-downscale/data/rtma/002_039/
gs://delos-downscale/data/rtma/002_040/
gs://delos-downscale/data/rtma/002_041/
gs://delos-downscale/data/rtma/002_042/
gs://delos-downscale/data/rtma/002_043/
gs://delos-downscale/data/rtma/002_044/
gs://delos-downscale/data/rtma/002_045/
gs://delos-downscale/data/rtma/002_046/
gs://delos-downscale/data/rtma/002_047/
gs://delos-downscale/data/rtma/002_048/
gs://delos-downscale/data/rtma/003_028/


In [ ]:
path = "./datasets/rtma/001_038/"

archives = [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.tgz')]
print(archives)

for archive in archives:
    print("Extracting:", archive)
    shutil.unpack_archive(archive, path)
    break

In [17]:
# .hdr and a .bsq file in each .tgz
hdr_files = glob.glob(os.path.join(path, '*.hdr'))
bsq_files = glob.glob(os.path.join(path, '*ws*.bsq'))

In [21]:
from pathlib import Path
file_dict={}
for bsq in bsq_files:
    print(bsq)
    with rio.open(bsq) as src:
        file_dict[Path(bsq).stem] = src.count
        src.close()

./datasets/rtma/001_038/ws_2013.bsq
./datasets/rtma/001_038/ws_2020.bsq
./datasets/rtma/001_038/ws_2021.bsq
./datasets/rtma/001_038/ws_2011.bsq
./datasets/rtma/001_038/ws_2014.bsq
./datasets/rtma/001_038/ws_2016.bsq
./datasets/rtma/001_038/ws_2017.bsq
./datasets/rtma/001_038/ws_2015.bsq
./datasets/rtma/001_038/ws_2018.bsq
./datasets/rtma/001_038/ws_2022.bsq
./datasets/rtma/001_038/ws_2023.bsq
./datasets/rtma/001_038/ws_2019.bsq
./datasets/rtma/001_038/ws_2012.bsq


In [26]:
from pprint import pprint
pprint(file_dict)

{'ws_2011': 10224,
 'ws_2012': 10944,
 'ws_2013': 10920,
 'ws_2014': 10920,
 'ws_2015': 10920,
 'ws_2016': 10944,
 'ws_2017': 10920,
 'ws_2018': 10920,
 'ws_2019': 10920,
 'ws_2020': 10944,
 'ws_2021': 10920,
 'ws_2022': 10920,
 'ws_2023': 6281}


In [ ]:
# weirdly have diff # of bands in each file, was expecting 365*24 bands if its a year's worth of hour observations.
# Val was able to confirm that this was intentional to model fires that started at end of year, so you have data to use for modeling spread into the next year
from collections import Counter
frequency_counter = Counter(list(file_dict.values()))
print(frequency_counter)

Counter({10920: 8, 10944: 3, 10224: 1, 6281: 1})


In [28]:
hours_over = [v-(24*365) for v in list(file_dict.values())] # hours over one year's worth of hourly observations
print("hours beyond one year",Counter(hours_over))
print("days beyond one year",Counter([v/24 for v in hours_over])) # how many days typically trying to extend

hours beyond one year Counter({2160: 8, 2184: 3, 1464: 1, -2479: 1})
days beyond one year Counter({90.0: 8, 91.0: 3, 61.0: 1, -103.29166666666667: 1})


In [50]:
with rio.open(bsq) as src:
    print(src.profile)
    arr = src.read()

{'driver': 'ENVI', 'dtype': 'float32', 'nodata': -9999.0, 'width': 40, 'height': 40, 'count': 10944, 'crs': CRS.from_wkt('PROJCS["NAD83 / Conus Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","5070"]]'), 'transform': Affine(1200.0, -0.0, -2362425.0,
       -0.0, -1200.0, 2049435.0), 'blockxsize': 40, 'blockysize': 1, 'tiled': False, 'interleave': 'band'}


In [ ]:
t = src.profile['transform'] # ux,y scale is 1200m, CRS linear unit is in degree
print(t)

| 1200.00,-0.00,-2362425.00|
|-0.00,-1200.00, 2049435.00|
| 0.00, 0.00, 1.00|


ERROR 10: Pointer 'hObject' is NULL in 'GDALGetMetadata'.

ERROR 10: Pointer 'hObject' is NULL in 'GDALGetMetadata'.

ERROR 10: Pointer 'hObject' is NULL in 'GDALGetMetadata'.



In [52]:
arr.shape

(10944, 40, 40)

In [ ]:
# parse into single-hour samples
import torch
t = torch.Tensor(arr)
t.shape
tt = torch.transpose(t,0,1)
tt.shape
torch.re

torch.Size([40, 10944, 40])

In [53]:
import numpy as np

# Example array with shape (3, 4, 5) representing (C, H, W)
my_array = np.random.rand(3, 4, 5) 
print("Original shape:", my_array.shape) 

# Add a batch dimension using expand_dims()
expanded_array = np.expand_dims(my_array, axis=0)
print("Expanded shape:", expanded_array.shape) 

# Example using reshape()
expanded_array_reshape = my_array.reshape((1, my_array.shape[0], my_array.shape[1], my_array.shape[2]))
print("Expanded shape using reshape():", expanded_array_reshape.shape)


Original shape: (3, 4, 5)
Expanded shape: (1, 3, 4, 5)
Expanded shape using reshape(): (1, 3, 4, 5)
